**Process:**
* Load and check data
* Data preprocess (2 datasets)
* Answer each question (7 in total)
 
**Notations:**

*stock.sas7bdat*
* PERMNO: a 5-digit firm identifier
* DATE: last trading day of the month, SAS format yymmddn8.
* COMNAM: company name
* EXCHCD: 1: NYSE, 2: AMEX, 3: NASDAQ
* SICCD: 4 digit SIC code
* PRC: closing price
* RET: monthly return
* SHROUT: shares outstanding in thousand shares

*ff3.sas7bdat*
* DATE: last trading day of the month
* SMB: small-minus-big return
* HML: high-minus-low return
* MKTRF: excess return on the market
* RF: risk-free rate
* UMD: momentum factor

# Load and check data

### 1. One argument of the existence of the CAPM alpha is the explanatory power of the CAPM, or lack thereof. You might have an excellent performance in your portfolio in the first homework because you take on higher risks that can only be captured by SMB or HML, or even UMD, but not by the market factor. Please examine whether your portfolio alpha is still significant with equation (1) and (2) to compare the intercept to the alpha with the CAPM. Use the original sample period to estimate the beta of the firms in homework 1 (2008-2012) and test for both high- and low-alpha portfolios and see if the alphas are still as large/small as they were. (Note: make a table for the parameters of different models and discuss.)

### 2. Which is the best model to explain your portfolio returns from 2008 to 2012? FF-3, Carhart 4-factor or the CAPM? (Note: you should judge the model performance by goodness-of-fit and parameter significance, not by alpha alone. Since these models are nested, please use F-tests to compare them.)

### 3. A lot of you might find the behavior of your portfolio coefficients are very different before and after the portfolio construction date (the end of 2012). Please use your best model at explaining your two portfolios to make your argument (whichever is better in the previous question should be used here) about the existence of a structural change with your portfolio (i.e. you have significant changes in regression parameters).

### 4. Please analyze the risk premium of your long-short strategy with these models. Show the parameters before and after the portfolio construction date (i.e. 2008-2012 and 2013-2017).

### 5. We often use the CAPM or FF-3 model or even four-factor model to estimate the cost-of-equity of a firm or to forecast the return on a security or a portfolio. Please use the portfolio return data from 2013 to 2017 for an out-of-sample test. Which model do you think is better for the job? To solve the problem, you need to calculate the prediction error by
![fromnt](https://i.imgur.com/AgcKjgL.png)
![back](https://i.imgur.com/hKJzgUE.png)

### 6. How do you examine if there exists collinearity between those risk factors in equation (2)? (Hint: there’s no collinearity) Please use 2008 to 2012 data to verify that.